## Imports and data loading

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.linalg import norm
from pysal.lib.weights import Queen
from spint.gravity import Gravity, Production
from spopt import MaxPHeuristic, RegionKMeansHeuristic, WardSpatial, Skater

N_CA_COUNTIES = 58
N_CA_TRACTS = 8057

In [2]:
def isCA_cts(x):
    # Vectorized evaluation if row is a CA to CA flow in SafeGraph data
    return [el[0] == '6' and len(el) == 10 for el in x]

In [3]:
ct_daily = pd.read_csv('../data/daily_ct2ct_08_10.csv', 
    converters={'geoid_o' : lambda x: str(x), 'geoid_d' : lambda x: str(x), 'visitor_flows' : lambda x: int(float(x)), 'pop_flows' : lambda x: int(float(x))}, 
    usecols=['geoid_o', 'geoid_d', 'lng_o', 'lat_o', 'lng_d', 'lat_d', 'visitor_flows', 'pop_flows'])
ct_daily = ct_daily[np.logical_and(isCA_cts(ct_daily['geoid_o']), isCA_cts(ct_daily['geoid_d']))]
ct_daily['geoid_o'] = '0' + ct_daily['geoid_o']
ct_daily['geoid_d'] = '0' + ct_daily['geoid_d']

CAtracts = gpd.read_file('../data/CAtract_level_data.csv', GEOM_POSSIBLE_NAMES="geometry", KEEP_GEOM_COLUMNS="NO")

In [21]:
# Convert cols to float
CAtracts['B00001_001E'] = pd.to_numeric(CAtracts['B00001_001E'])
CAtracts['B02001_002E'] = pd.to_numeric(CAtracts['B02001_002E'])
CAtracts['B07011_001E'] = pd.to_numeric(CAtracts['B07011_001E'])
CAtracts.dropna(inplace=True)

In [22]:
Wt = Queen.from_dataframe(CAtracts)

/home/thoffman/anaconda3/envs/gds/lib/python3.6/site-packages/pysal/lib/weights/weights.py:170: UserWarning: The weights matrix is not fully connected. There are 3 components
  warnings.warn("The weights matrix is not fully connected. There are %d components" % self.n_components)


## Main sweeper function

In [28]:
def regsweep(method='ward', n_clusters=N_CA_TRACTS/2, attrs=['B07011_001E']):
    # Set up and run model 
    if method == 'ward':
        model = WardSpatial(CAtracts, Wt, attrs, n_clusters=n_clusters)
    elif method == 'skater':
        model = Skater(CAtracts, Wt, attrs, n_clusters=n_clusters)
    else: raise RuntimeError('enter \'ward\' or \'skater\', other regionalizations unimplemented')
    model.solve()
    CAtracts['labels'] = model.labels_
    
    # Aggregate data and flows by the new regionalization
    regtdata = CAtracts.dissolve(by='labels', aggfunc='mean')
    CAtdata = ct_daily.join(CAtracts.set_index('GEOID'), on='geoid_o')
    
    # function to determine what region a census tract is in
    regionof = lambda geoid : CAtdata.loc[CAtdata['geoid_o'] == geoid, 'labels'].iloc[0]
    
    # Manually aggregate flows -- SLOW
    flows = np.zeros((CAtdata['labels'].unique().shape[0], CAtdata['labels'].unique().shape[0]))
    for o in range(flows.shape[0]):
        # Locate all flows origininating in region i and going to different regions    
        for r in CAtdata.loc[CAtdata['labels'] == o].iterrows():
            dest = regionof(r[1]['geoid_o'])
            if o != dest: flows[o, dest] += r[1]['pop_flows']
                
    # use aggregated centroids to create costs in Euclidean distances (do this with a spatial weight?)
    coords = np.hstack((regtdata.centroid.x.values.reshape(-1, 1), regtdata.centroid.y.values.reshape(-1, 1)))
    o_coords = np.repeat(coords, coords.shape[0], axis=0)
    d_coords = np.tile(coords, (coords.shape[0], 1))
    cost = norm(o_coords - d_coords, axis=1)
    
    # Set up variables and fit gravity model
    basevars = regtdata[['B00001_001E', 'B02001_002E']].astype(int).values
    o_vars = np.repeat(basevars, basevars.shape[0], axis=0)
    d_vars = np.tile(basevars, (basevars.shape[0], 1))
    
    iflows = flows.astype(int).flatten()
    spintmodel = Gravity(iflows, o_vars, d_vars, cost, 'exp')
    
    # Compute how average area per unit (proxy for amount of aggregation)
    avgarea = regdata.area.mean()
    
    return spintmodel.AIC, spintmodel.SRMSE, spintmodel.pseudoR2, avgarea

## Sweep!

We average AIC and SRMSE over `r` trials for `n_clusters` between 4 and `N_CA_TRACTS` (8057). (Spatial interaction doesn't make sense if there's only one unit and `Gravity` requires more rows than columns in the estimation.)

In [29]:
r = 25
method = 'ward'
w_aic_arr = np.zeros((N_CA_TRACTS - 4 + 1, 1))
w_srmse_arr = np.zeros((N_CA_TRACTS - 4 + 1, 1))
w_pr2_arr = np.zeros((N_CA_TRACTS - 4 + 1, 1))
w_avgarea_arr = np.zeros((N_CA_TRACTS - 4 + 1, 1))

for k in tqdm(range(4, N_CA_TRACTS+1)):
    for itr in range(r):
        aic, srmse, pr2, avgarea = regsweep(method=method, n_clusters=k)
        w_aic_arr[k-4] += aic/r
        w_srmse_arr[k-4] += srmse/r
        w_pr2_arr[k-4] += pr2/r
        w_avgarea_arr[k-4] += avgarea/r

  0%|          | 0/8054 [00:00<?, ?it/s]/home/thoffman/anaconda3/envs/gds/lib/python3.6/site-packages/sklearn/cluster/_agglomerative.py:247: UserWarning: the number of connected components of the connectivity matrix is 3 > 1. Completing it to avoid stopping the tree early.
  affinity='euclidean')
  0%|          | 0/8054 [11:12<?, ?it/s]


KeyboardInterrupt: 